In [2]:
import os
os.sys.path

['/gpfs/workdir/liashuhamy/clr',
 '/gpfs/softs/languages/anaconda3/2020.02/lib/python37.zip',
 '/gpfs/softs/languages/anaconda3/2020.02/lib/python3.7',
 '/gpfs/softs/languages/anaconda3/2020.02/lib/python3.7/lib-dynload',
 '',
 '/gpfs/users/liashuhamy/.local/lib/python3.7/site-packages',
 '/gpfs/softs/languages/anaconda3/2020.02/lib/python3.7/site-packages',
 '/gpfs/softs/languages/anaconda3/2020.02/lib/python3.7/site-packages/IPython/extensions',
 '/gpfs/users/liashuhamy/.ipython']

In [3]:
os.sys.path.append('/gpfs/users/liashuhamy/.conda/envs/pcamenv/lib/python3.7/site-packages')

In [4]:
import numpy as np
# import cv2

In [5]:

import tensorflow as tf
print(tf.__version__)

2.3.2


In [6]:
!nvidia-smi

Mon Jun 28 18:56:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 440.44       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro RTX 5000     Off  | 00000000:2F:00.0 Off |                  Off |
| 33%   33C    P2    47W / 230W |    394MiB / 16125MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import matplotlib.pyplot as plt
# from imutils import paths
from tqdm import tqdm
import tensorflow as tf
import seaborn as sns
import numpy as np
# import cv2

# Random seed fixation
tf.random.set_seed(666)
np.random.seed(666)

In [12]:
# !pip install imutils

In [8]:
import h5py

X_train = h5py.File('/gpfs/workdir/liashuhamy/camel/raw/pre_9_no_norm_patient_split/camelyonpatch_level_2_split_train_x.h5', 'r')['x'][:]
y_train = h5py.File('/gpfs/workdir/liashuhamy/camel/raw/pre_9_no_norm_patient_split/camelyonpatch_level_2_split_train_y.h5', 'r')['y'][:].reshape((-1,1))

X_test = h5py.File('/gpfs/workdir/liashuhamy/camel/raw/pre_9_no_norm_patient_split/camelyonpatch_level_2_split_valid_x.h5', 'r')['x'][:]
y_test = h5py.File('/gpfs/workdir/liashuhamy/camel/raw/pre_9_no_norm_patient_split/camelyonpatch_level_2_split_valid_y.h5', 'r')['y'][:].reshape((-1,1))

In [9]:
y_train

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=uint8)

In [10]:
X_train.shape, y_train.shape

((95106, 256, 256, 3), (95106, 1))

In [11]:
X_test.shape, y_test.shape

((30296, 256, 256, 3), (30296, 1))

In [12]:
np.sum(y_test)

15148

In [13]:
from PIL import Image
def resize(img, size):
    img = Image.fromarray(img)
    img = img.resize(size)
    return np.array(img)

In [15]:
import cv2
import numpy as np
new_x = []
from tqdm import tqdm
for i in tqdm(range(X_train.shape[0])):
    new_x.append(cv2.resize(X_train[i], (224, 224)))
X_train = np.array(new_x)

new_x = []
from tqdm import tqdm
for i in tqdm(range(X_test.shape[0])):
    new_x.append(cv2.resize(X_test[i], (224, 224)))
X_test = np.array(new_x)

X_train.shape, X_test.shape

100%|██████████| 30296/30296 [00:04<00:00, 7275.56it/s]


((95106, 224, 224, 3), (30296, 224, 224, 3))

In [ ]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [16]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

/gpfs/softs/languages/anaconda3/2020.02/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/gpfs/softs/languages/anaconda3/2020.02/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
y_train_enc = np.array([[1-i, i] for i in y_train]).reshape((-1,2))
y_test_enc = np.array([[1-i, i] for i in y_test]).reshape((-1,2))

In [18]:
y_train_enc.shape, y_test_enc.shape

((95106, 2), (30296, 2))

In [19]:
y_test_enc[:3], y_test[:3]

(array([[1, 0],
        [0, 1],
        [0, 1]], dtype=uint8),
 array([[0],
        [1],
        [1]], dtype=uint8))

In [22]:
from classification_models.tfkeras import Classifiers
def get_resnet_simclr(hidden_1, hidden_2, hidden_3):
    ResNet18, preprocess_input = Classifiers.get('resnet18')
    base_model = ResNet18(include_top=False, weights=None, input_shape=(224, 224, 3))
    base_model.trainable = True
    inputs = Input((224, 224, 3))
    h = base_model(inputs, training=False)
    h = GlobalAveragePooling2D()(h)

    projection_1 = Dense(hidden_1)(h)
#     projection_1 = Activation("relu")(projection_1)
#     projection_2 = Dense(hidden_2)(projection_1)
#     projection_2 = Activation("relu")(projection_2)
#     projection_3 = Dense(hidden_3)(projection_2)

    resnet_simclr = Model(inputs, projection_1)

    return resnet_simclr

In [36]:

# resnet_simclr = get_resnet_simclr(256, 128, 50)
# resnet_simclr.load_weights('model_18.h5')

ResNet18, preprocess_input = Classifiers.get('resnet18')
base_model = ResNet18(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = True
inputs = Input((224, 224, 3))
h = base_model(inputs, training=False)
h = GlobalAveragePooling2D()(h)
resnet_simclr = Model(inputs, h)

resnet_simclr.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
functional_7 (Functional)    (None, 7, 7, 512)         11186889  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
Total params: 11,186,889
Trainable params: 11,178,947
Non-trainable params: 7,942
_________________________________________________________________


In [37]:
def plot_training(H):
	with plt.xkcd():
# 		plt.plot(H.history["loss"], label="train_loss")
# 		plt.plot(H.history["val_loss"], label="val_loss")
		plt.plot(H.history["accuracy"], label="train_acc")
		plt.plot(H.history["val_accuracy"], label="val_acc")
		plt.title("Training Accuracy")
		plt.xlabel("Epoch #")
		plt.ylabel("Accuracy")
		plt.legend(loc="lower left")
		plt.show()

In [38]:

def get_linear_model(features):
    linear_model = Sequential([Dense(2, input_shape=(features, ), activation="softmax")])
    return linear_model

In [39]:
resnet_simclr.layers[1].trainable = False
resnet_simclr.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
functional_7 (Functional)    (None, 7, 7, 512)         11186889  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
Total params: 11,186,889
Trainable params: 0
Non-trainable params: 11,186,889
_________________________________________________________________


In [40]:
# Encoder model with non-linear projections
projection = Model(resnet_simclr.input, resnet_simclr.layers[-1].output)

import time
tic = time.time()
# Extract train and test features
train_features = projection.predict(X_train)
test_features = projection.predict(X_test)
print(time.time() - tic)

print(train_features.shape, test_features.shape)

75.67427253723145
(95106, 512) (30296, 512)


In [41]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# train_features = scaler.fit_transform(train_features)
# test_features = scaler.transform(test_features)

In [42]:
# Early Stopping to prevent overfitting
es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=2, restore_best_weights=True)

In [43]:
# !pip install 'gast==0.2.2'

In [44]:
train_features.shape

(95106, 512)

In [46]:
linear_model = get_linear_model(512)
linear_model.summary()
linear_model.compile(loss="categorical_crossentropy", metrics=["accuracy"],
                     optimizer="adam")
history = linear_model.fit(train_features, y_train_enc,
                 validation_data=(test_features, y_test_enc),
                 batch_size=64,
                 epochs=500,
                 callbacks=[es])
plot_training(history)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 2)                 1026      
Total params: 1,026
Trainable params: 1,026
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
1487/1487 [==============================] - 6s 4ms/step - loss: 0.1868 - accuracy: 0.9317 - val_loss: 0.4149 - val_accuracy: 0.8472
Epoch 2/500
  44/1487 [..............................] - ETA: 3s - loss: 0.1654 - accuracy: 0.9393

KeyboardInterrupt: 

In [ ]:
y_train_enc

In [47]:
from sklearn.linear_model import LinearRegression, SGDClassifier
reg = SGDClassifier().fit(train_features, y_train)
reg.score(test_features, y_test)

/gpfs/softs/languages/anaconda3/2020.02/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.857241880116187

In [103]:
X_train.shape

(95106, 224, 224, 3)

In [68]:
train_features.shape

(1569, 128)

In [48]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, max_iter=1000).fit(train_features, y_train)

/gpfs/softs/languages/anaconda3/2020.02/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [49]:
clf.score(test_features, y_test)

0.865163717982572